In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
data_root = os.path.join('.', 'notMNIST_data') # Change me to store data elsewhere
pickle_file = os.path.join(data_root, 'notMNIST.pickle')

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [9]:
save_dir = os.path.join(data_root, 'save_v3')

new_graph = tf.Graph()
with tf.Session(graph=new_graph) as sess:
    # Import the previously export meta graph.
    # saver = tf.train.import_meta_graph(tf.train.latest_checkpoint(os.getcwd())+".meta")
    # saver.restore(sess, tf.train.latest_checkpoint(os.getcwd()))
    saver = tf.train.import_meta_graph(save_dir + '/my-model-new-3000.meta')
    saver.restore(sess, save_dir +  '/my-model-new-3000')
    print('Model restored.')
    # W_conv1_trained = sess.run('Variable_2:0')
    # plt.figure(1)
    # for i in np.arange(32):
    #    plt.subplot(4,8,i+1)
    #    plt.imshow(W_conv1_trained[:,:,0,i])
    #    plt.gray()
    # plt.show()
    graph = tf.get_default_graph()
    ops = graph.get_operations()
    print(len(ops))
    # print(graph.get_operations())
    # for op in ops:
    #     print(op.name)
    correct_prediction = graph.get_tensor_by_name('accuracy/correct_prediction/Equal:0')
    accuracy = graph.get_tensor_by_name('accuracy/accuracy/Mean:0')
    
    correct_valid, accuracy_valid = sess.run([correct_prediction, accuracy], feed_dict={'input/input_data:0': valid_dataset,
                                            "input/input_labels:0": valid_labels})
    print('valid accuracy %g' % accuracy_valid)
    # op1 = graph.get_operation_by_name("Equal")
    # print(op1)
    # print(op1.name)
    # print(mnist.train.labels[:1])
    # print(op1.values())
    # print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

INFO:tensorflow:Restoring parameters from ./notMNIST_data/save_v3/my-model-new-3000
Model restored.
851
valid accuracy 0.8599


In [42]:
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
for i in np.arange(num_labels):
    labels_total = np.sum(np.argmax(valid_labels, axis=1) == i)
    labels_correct = np.sum(np.argmax(valid_labels[correct_valid], axis=1) == i)
    print('Label: %s. Accuracy: %f.' % (labels[i], labels_correct/labels_total))

Label: A. Accuracy: 0.853000.
Label: B. Accuracy: 0.827000.
Label: C. Accuracy: 0.871000.
Label: D. Accuracy: 0.892000.
Label: E. Accuracy: 0.860000.
Label: F. Accuracy: 0.889000.
Label: G. Accuracy: 0.868000.
Label: H. Accuracy: 0.867000.
Label: I. Accuracy: 0.821000.
Label: J. Accuracy: 0.851000.
